In [6]:
ljy_fp = r"D:\workspace\hw-memory\benchmarks\final_benchmark\spec_benchmark.asm"
GuGuJi_fp = r"D:\workspace\hw-memory\benchmarks\final_benchmark\spec_benchmark.asm"
GuGuJi_spec_fp = r"D:\workspace\hw-memory\benchmarks\final_benchmark\spec_benchmark.asm"
GuGuJi_oldfp = r"/benchmarks/old_benchmark/spec_example/spec2006_470.lbm/lbm_part.asm"
fp = ljy_fp
guguji_segfp = r"D:\workspace\hw-memory\benchmarks\final_benchmark\spec_benchmarkD.asm"
seg_fp = r'D:\workspace\hw-memory\benchmarks\final_benchmark\spec_benchmarkD.asm'


In [7]:
'''读指令, 指令对象化'''
from newCFG.cfg import proc_identify
from newCFG.isa import Instruction, AddrMode
from typing import Tuple
from newCFG.read_asm import AsmFileReader, StatementType
from cache_analysis.read_segment import segmentReader

reader =AsmFileReader(fp)
segreader = segmentReader(seg_fp)

statements = list()
temp_stat = list()
for s in reader.statements:
    s: Tuple[StatementType, tuple]
    if s[0] == StatementType.Instruction:
        statements.append((s[0], Instruction(s[1])))
        temp_stat.append(Instruction(s[1]))
    elif s[0] == StatementType.SubProcedure:
        statements.append(s)

        

for i in temp_stat:
     if i.is_ls:
        print(i.tokens,i.ls_reg_target,i.ls_reg_target_list,i.ls_addr_offset)

procs = proc_identify(statements)
for proc in procs:
    print(proc.name, proc.beg_addr.hex_str())
    if False and proc.name == 'main':
        for inst in proc.instruction:
            print(inst.addr.hex_str(), inst.opcode, inst.name, inst.operands, inst.branch_info)
print()

In [8]:
'''分proc'''
from graphviz import Digraph
from newCFG.cfg import draw_proc, find_cycle, has_cycle, proc_draw_edges

proc_draw_edges(procs)

is_cycle = has_cycle(procs)
if is_cycle:
    c = [p.name for p in find_cycle(procs)]
    raise RuntimeError("Loop between procedures is not allowed: {}.".format(c))

g: Digraph = draw_proc(procs)
g.render(filename='procedures', directory='./output', format='svg')

In [9]:
'''处理call_graph'''
from newCFG.cfg import CallGraph

call_graph = CallGraph(procs)

g = call_graph.draw_graph()
g.render(filename='call_graph', directory='./output', format='svg')

In [10]:
'''生成TCFG和loop信息'''
from newCFG.cfg import TCfg


tcfg = TCfg(call_graph)
tcfg.build_tcfg()

tcfg_nodes = tcfg.all_nodes
tcfg_edges = tcfg.edges

g = tcfg.draw_graph()
g.render(filename='tcfg', directory='./output', format='svg')

#for node in tcfg_nodes:
#    node.set_rw_condition()
#    #print(node.name, node.base_proc.name, node.inst_range)
#    print(node.name, node.base_proc.name)
#    print("出边")
#    for i in node.outgoing_edge:
#        print(i.dst.name)
#    print("入边")
#    for i in node.incoming_edge:
#        print(i.src.name)
#
#    print()
tcfg.build_loop_hrchy()
tcfg.add_loop_bound(r'D:\workspace\hw-memory\benchmarks\loop_bound.txt')
for l in tcfg.loops:
    print("{} (in {} children {}): {}->{} {}".format(l.name, l.father.name if l.father is not None else "None", [sub_l.name for sub_l in l.children], l.back_edge.src.name, l.back_edge.dst.name, [n.name for n in l.all_nodes]))
    l.back_edge.is_backEdge = True
for n in tcfg.all_nodes:
    temp = n.inside_loop.name if n.inside_loop is not None else 'None'
    print("{} {}".format(n.name, temp))



#for n in tcfg_nodes:
#    print(n.name,"n的值",n.node_value)
        

#for e in tcfg_edges:
#    print(e.src.name,e.dst.name,e.is_backEdge)




In [11]:
from rwcond_out import loadstore_Obj
lds_obj = loadstore_Obj(segreader,tcfg)
lsproc = lds_obj.lsproc
ls_loop_info = lds_obj.loop_info

for k,v in ls_loop_info.items():
    print(k,v)

In [12]:
# '''设置node和edge上的的value'''
# import random
# for l in tcfg.loops:
#     l.back_edge.is_backEdge = True
# for n in tcfg_nodes:
#     n.set_rw_data()

# temp_bool = True

# while temp_bool:
#     no_end = False

#     for n in tcfg_nodes:
#         n.set_rw_value()
#         if not n.no_out:
#             no_end = True
#         temp_bool = no_end

# for e in tcfg_edges:
#     if e.is_backEdge:
#         e.loop_value = random.randint(0,10)
#         e.edge_value = 2

# #for e in tcfg_edges:
# #    print(e.src.name,e.dst.name,e.edge_value)

In [13]:
# '''ls分析和读写特征'''

# from newCFG.cfg import TCfg
# from newCFG.loadstore import LSProc
# from newCFG.rw_condition import RWProc, RWType

# lsproc = LSProc(tcfg_nodes)

# lds_table = lsproc.ls_table

# for i in lds_table:
#     i.final_addr
#     i.local_offset

# #for i in lds_table:
# #    if i.target_num == 1:
#  #       pass
#         #print(i.ins.tokens,i.reg_target,i.addr_offset) 
#  #   else:
#         #print(i.ins.tokens,i.reg_target_list[0],i.reg_target_list[1],i.addr_offset,i.final_addr)

# rwproc = RWProc(lds_table)

# Global_Tolerant_value = 0
# Global_Intolerant_value = 0

# for rwu in rwproc.rw_table:
#     #print(rwu.ins.tokens,rwu.find_cycle,rwu.ins.final_addr,rwu.is_torrent)
#     if rwu.is_torrent == RWType.Global_Tolerant:
#         Global_Tolerant_value += rwu.find_cycle
#         print(rwu.ins.tokens,rwu.ins.final_addr,rwu.node.name)
# print()
# for rwu in rwproc.rw_table:
#     #print(rwu.ins.tokens,rwu.find_cycle,rwu.ins.final_addr,rwu.is_torrent)
#     if rwu.is_torrent == RWType.Global_Intolerant:
#         Global_Intolerant_value += rwu.find_cycle
#         print(rwu.ins.tokens,rwu.ins.final_addr,rwu.node.name)

# print("全局的容错路径为：",Global_Tolerant_value)
# print("全局的非容错路径为：",Global_Intolerant_value)   

In [14]:
# '''cache分析'''

# #打印给cache分许需要的数据
# for i in lds_table:
#     print("指令",i.ins.tokens,"是否是SP",i.is_sp,"指令地址",i.ins.addr.val(),"访存地址",i.final_addr,"数据宽度",i.ins.ls_data_width,"是否是加入一个数组",i.ins.is_data_group)


In [15]:
from newCFG.heat_analysis import loop_heat
test=loop_heat(tcfg,lsproc,r'C:\Users\13377\Desktop\hw-memory-master (5)\hw-memory-master\benchmarks\final_benchmark\spec_benchD.asm')
test.do_it()


In [ ]:

# TODO.


In [ ]:

# from newCFG.rw_condition import RWProc, RWType
# from newCFG.rw_condition_out import RWOut_Proc



# rwout = RWOut_Proc(tcfg_nodes,segreader,rwproc.rw_table,tcfg.loops)




